<a href="https://colab.research.google.com/github/m0hit-kumar/RASA/blob/main/CollabNotebook/RASA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.6 MB/s eta 0:00:00


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

model_name = "facebook/bart-base"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [ ]:
!ls
!unzip data.zip

 BombBlast.txt	 earthquake.txt   first_data.txt    sample_data   Volcano.txt
 Downloads.zip	 falseData.txt	 'processed data'   Tornado.txt   War.txt
Archive:  Downloads.zip
replace War.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#step 1 reading files and vonverting txt to json
import json
def txt_to_json(filename):
  with open(filename+".txt", "r", encoding="utf-8") as file:
    json_text = file.read()
  json_data = json.loads(json_text)
  return json_data

In [ ]:
#step 2 summarizing text
test_data = """
    The United States of America, commonly known as the United States (U.S. or US), is a country primarily located
    in North America. It consists of 50 states, a federal district, five major unincorporated territories, 186
    Indian reservations, and some minor possessions. The capital city is Washington, D.C., and the most populous
    city is New York City.
"""

def summarize(input_text):
  inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
  summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return summary


In [ ]:

def summarize_text(texts, model, tokenizer, max_length=150, num_beams=4, length_penalty=2.0):
    # Tokenize and encode the texts in batches
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length, verbose=False)

    # Generate summaries
    summaries = []
    for i in range(len(texts)):
        input_ids = inputs["input_ids"][i].unsqueeze(0).to(model.device)
        attention_mask = inputs["attention_mask"][i].unsqueeze(0).to(model.device)

        summary_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            min_length=10,
            length_penalty=length_penalty,
            num_beams=num_beams,
            early_stopping=True
        )

        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    return summaries


In [ ]:
#step 3 converting into rasa data convtor format
def data_to_json(key,data_list):
  data=[]
  for i in data_list:
    x={
          "text": i,
          "intent": key
    }
    data.append(x)

  return data

In [ ]:
#step 4  Write json file data file
import yaml
def write_into_json(full_data):
  json_data = {
    "rasa_nlu_data": {
    "common_examples": full_data
    }
  }
  json_string = json.dumps(json_data, indent=4)
  with open("processedData", "w") as output_file:
    output_file.write(json_string)


In [ ]:
#pro function

def pro_function(filename):
  json_data=txt_to_json(filename)
  news=[i["text"] for i in json_data["news"]]
  # summarized_news=[summarize(i) for i in news]
  summarized_news=summarize_text(news,model, tokenizer)
  json_summarized_data=data_to_json(filename,summarized_news)


In [ ]:
#pro max function
def pro_max_function(file_list):
  full_data_variable_list=[pro_function(i) for i in file_list]
  full_data= [item for sublist in full_data_variable_list for item in sublist]
  write_into_json(full_data)

In [ ]:
file_list=["War","Tornado","BombBlast","falseData","Volcano"] #change with respect to file name

In [ ]:
pro_max_function(file_list)   #excute to destroy half of the data chunk

In [ ]:
#step to use file
# 1) go to pc create env
# 2) activate env
# 3) install rasa <pip install rasa>
# 4) run command in root directory <rasa init>
# 5) rasa data convert <rasa data convert nlu --data processedData.json>    note: run command in the folder where u have data otherwise change the path accordingly
# 6) rename the generated file to nlu.yaml and place it in the data folder
# 7) run command rasa train
# 8) wala! you gets error. if not your model magicallly appread in models folder
# 9) then delete the model and get some coffe/tea acc. to prefrece
# 10) get a good life what are u doing, sab doglapan hai
# 11) if you are smart and didnt deleted the model you can <run rasa> in terminal to test the model
